# Script Molde

Cria os moldes a partir dos quais as requisições do CEPESPData serão comparadas

In [1]:
import numpy as np
import pandas as pd
import os
import csv

if not os.path.exists("data/moldes"):
    os.makedirs("data/moldes")

## Molde Vagas

In [2]:
# Readind vagas paths and storing
b_path = "raw_data/arquivos_vagas/unziped/"
paths = {str(ano): (lambda ano: os.listdir(b_path + str(ano)))(ano) for ano in range(1998,2022,4)}

# Defining Headers
headers = ["DATA_GERACAO", "HORA_GERACAO", "ANO_ELEICAO","DESCRICAO_ELEICAO", "SIGLA_UF",
           "SIGLA_UE", "NOME_UE", "CODIGO_CARGO", "DESCRICAO_CARGO", "QTDE_VAGAS"]

In [3]:
data = []
for ano in range(1998,2022,4):
    ano = str(ano)
    print("Reading {}...".format(ano))
    if ano == '2014' or ano == '2018':
        skiprows_use=1
    else:
        skiprows_use=None

    for file in paths[ano]:
        if 'txt' in file or 'csv' in file:
            data.append(pd.read_csv(b_path + ano + "/" + file,
                                sep = ';',
                                encoding='latin_1',
                                header=None,
                                skiprows=skiprows_use))        

Reading 1998...
Reading 2002...
Reading 2006...
Reading 2010...
Reading 2014...
Reading 2018...


In [4]:
for i in range(len(data)):
        if len(data[i].columns) == 10:
            data[i].columns = headers
        else:
            data[i] = data[i].iloc[:,[0,1,2,6,9,10,11,12,13,14]]
            data[i].columns = headers
    
# Bindng all data
data = pd.concat(data)

# Droping obs
data = data[np.in1d(data.DESCRICAO_ELEICAO, ['Eleição Suplementar Governador AM', 'Eleição Suplementar Governador - TO'], invert=True)]

# Droping coluns which will not be used
data = data.iloc[:,2:len(data.columns)].drop(["SIGLA_UF", 'DESCRICAO_ELEICAO'], axis = 1)

# Creating by hand president DataFrame
vagas_presidente = {'ANO_ELEICAO': np.arange(1998,2018,4),
                    'SIGLA_UE': "BR",
                    'CODIGO_CARGO': 1,
                    'QTDE_VAGAS': 1}

In [5]:
data = data.apply(lambda x: x.str.upper() if hasattr(x, 'str') else x, axis=0)

In [6]:
data = data[['ANO_ELEICAO', 'SIGLA_UE', 'CODIGO_CARGO', "QTDE_VAGAS"]]

In [7]:
presidente_df = pd.DataFrame(vagas_presidente)
    
# Binding president and data together

data = pd.concat([data, presidente_df])

data = data.drop_duplicates()

print("Writing data into molde_vagas.csv")

data.to_csv("data/moldes/molde_vagas.csv")

Writing data into molde_vagas.csv


## Molde Seção

In [8]:
# Readind vagas paths and storing
b_path = "raw_data/arquivos_secao/unziped/"
paths = {str(ano): (lambda ano: os.listdir(b_path + str(ano)))(ano) for ano in range(1998,2022,4)}


# Defining Headers
headers = ["DATA_GERACAO", "HORA_GERACAO", "ANO_ELEICAO", "NUM_TURNO", "DESCRICAO_ELEICAO",
           "SIGLA_UF", "SIGLA_UE", "CODIGO_MUNICIPIO", "NOME_MUNICIPIO","NUM_ZONA","NUM_SECAO",
           "CODIGO_CARGO", "DESCRICAO_CARGO", "QTD_APTOS", "QTD_COMPARECIMENTO", "QTD_ABSTENCOES", 
           "QT_VOTOS_NOMINAIS", "QT_VOTOS_BRANCOS", "QT_VOTOS_NULOS", "QT_VOTOS_LEGENDA",
           "QT_VOTOS_ANULADOS_APU_SEP"]

headers_2018 = ['DATA_GERACAO', 'HORA_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
                'NM_TIPO_ELEICAO', 'NUM_TURNO', 'CD_ELEICAO', 'DESCRICAO_ELEICAO', 'DT_ELEICAO',
                'TP_ABRANGENCIA', 'SIGLA_UF', 'SIGLA_UE', 'NM_UE', 'CODIGO_MUNICIPIO',
                'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'CODIGO_CARGO', 'DS_CARGO',
                'QTD_APTOS', 'QTD_COMPARECIMENTO', 'QTD_ABSTENCOES', 'QT_VOTOS_NOMINAIS',
                'QT_VOTOS_BRANCOS', 'QT_VOTOS_NULOS', 'QT_VOTOS_LEGENDA',
                'QT_VOTOS_PENDENTES']


In [9]:
a = pd.read_csv("raw_data/arquivos_secao/unziped/2018/detalhe_votacao_secao_2018_TO.csv", sep = ';', encoding='latin-1')

In [10]:
data = []
for ano in range(2006,2022,4):
    ano = str(ano)
    print("Parsing data from {}...".format(ano))
    data_spec = []
    if ano == '2018':
        header_use = headers_2018
        skiprows_use = 1
    else:
        header_use = headers
        skiprows_use = None
        
    for file in paths[ano]:
        if 'txt' in file or 'csv' in file:
            banco_temp = pd.read_csv(b_path + ano + '/' + file,
                                     sep = ';',
                                     header=None,
                                     names = header_use,
                                     skiprows = skiprows_use,
                                     encoding="latin-1")
            
            banco_temp = banco_temp[['ANO_ELEICAO',
                                     'NUM_TURNO',
                                     'SIGLA_UE',
                                     'CODIGO_MUNICIPIO',
                                     'CODIGO_CARGO',
                                     'QTD_APTOS',
                                     'QTD_COMPARECIMENTO']]
            data_spec.append(banco_temp)

    data_spec = pd.concat(data_spec)

    data_spec = data_spec.groupby(['ANO_ELEICAO',
                                   'NUM_TURNO',
                                   'SIGLA_UE',
                                   'CODIGO_MUNICIPIO',
                                   'CODIGO_CARGO',])

    data_spec = data_spec.sum().reset_index()

    data.append(data_spec)

data = pd.concat(data)

Parsing data from 2006...
Parsing data from 2010...
Parsing data from 2014...
Parsing data from 2018...


In [11]:
for ano in np.arange(2010,1998,-8):
    temp = data.loc[(data.ANO_ELEICAO == ano) & (data.CODIGO_CARGO == 5),["QTD_APTOS", "QTD_COMPARECIMENTO"]]
    temp = temp * 2
    data.loc[(data.ANO_ELEICAO == ano) & (data.CODIGO_CARGO == 5),["QTD_APTOS", "QTD_COMPARECIMENTO"]] = temp

temp = data.loc[(data.ANO_ELEICAO == 2002) & (data.CODIGO_CARGO == 1),["QTD_APTOS", "QTD_COMPARECIMENTO"]]

temp = temp / 2

data.loc[(data.ANO_ELEICAO == 2002) & (data.CODIGO_CARGO == 1),["QTD_APTOS", "QTD_COMPARECIMENTO"]] = temp

data.to_csv("data/moldes/molde_secao.csv")    

In [13]:
data

,ANO_ELEICAO,NUM_TURNO,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,QTD_APTOS,QTD_COMPARECIMENTO
0,2006,1,AC,1007,1,5140,4297
1,2006,1,AC,1007,3,5140,4297
2,2006,1,AC,1007,5,5140,4297
3,2006,1,AC,1007,6,5140,4297
4,2006,1,AC,1007,7,5140,4297
5,2006,1,AC,1015,1,4983,3974
6,2006,1,AC,1015,3,4983,3974
7,2006,1,AC,1015,5,4983,3974
8,2006,1,AC,1015,6,4983,3974
9,2006,1,AC,1015,7,4983,3974
